# ETL Process to clean data before analysis

## Preparing environment

In [348]:
import pandas as pd

In [349]:
import sys
sys.path.append('../high_performance_employee_resign_prediction')
from utils import paths

## Data Dictionary

* id_employee - Employee ID
* id_last_boss - Boss ID
* seniority - 1: For people who don't have anyone in charge, 2: For those who have people in charge
* work_modality - Work modality specified on employee contract
* office_distance -  Distance in kilometers from employee home to work.
* low_health_days -  Number of days pf justified sick leave since employee started to work for the company.
* gender - Employee gender
* recruitment_channel - Indicates the means by which the application process took place. 
* average_permanence - Mean time in years that the employee has worked in previous laboral experience.
* birth_date - Birth date in format dd/mm/yyyy.
* salary - It is the amount corresponding to the monthly remuneration received.
* psi_score - Is the score obtained in psychometric test during selection process.
* join_date - Date when the employee started to work for the company.
* marital_estatus - Marital Estatus.
* performance_score - Is the score obtained by the employee it his last quarterly performance test if he had.
* resign - "0" if employee continues working or "1" if employee left the company on his first 6 months.

## Extracting data

In [350]:
train_df = pd.read_csv(paths.data_raw_dir('train_data.csv'), parse_dates=['birth_date', 'join_date'], sep=';')

test_df = pd.read_csv(paths.data_raw_dir('test_data.csv'), parse_dates=['birth_date', 'join_date'], sep=';')

C:\Users\WILLIAM\AppData\Local\Temp\ipykernel_17532\388407031.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  train_df = pd.read_csv(paths.data_raw_dir('train_data.csv'), parse_dates=['birth_date', 'join_date'], sep=';')
C:\Users\WILLIAM\AppData\Local\Temp\ipykernel_17532\388407031.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  train_df = pd.read_csv(paths.data_raw_dir('train_data.csv'), parse_dates=['birth_date', 'join_date'], sep=';')
C:\Users\WILLIAM\AppData\Local\Temp\ipykernel_17532\388407031.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  test_df = pd.read_csv(paths.data_raw_dir('test_data.csv'), parse_dates=['birth_date',

## Checking data

In [351]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152 entries, 0 to 2151
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_employee          2152 non-null   int64         
 1   id_last_boss         2061 non-null   float64       
 2   seniority            2152 non-null   int64         
 3   work_modality        2152 non-null   object        
 4   office_distance      2152 non-null   float64       
 5   low_health_days      2152 non-null   int64         
 6   gender               2152 non-null   object        
 7   recruitment_channel  2152 non-null   object        
 8   average_permanence   2152 non-null   int64         
 9   birth_date           2152 non-null   datetime64[ns]
 10  salary               2152 non-null   int64         
 11  performance_score    2084 non-null   float64       
 12  psi_score            2152 non-null   int64         
 13  join_date            2152 non-nul

In [352]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020 entries, 0 to 2019
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_employee          2020 non-null   int64         
 1   id_last_boss         1937 non-null   float64       
 2   seniority            2020 non-null   int64         
 3   work_modality        2020 non-null   object        
 4   office_distance      2020 non-null   float64       
 5   low_health_days      2020 non-null   int64         
 6   gender               2020 non-null   object        
 7   recruitment_channel  2020 non-null   object        
 8   average_permanence   2020 non-null   int64         
 9   birth_date           2020 non-null   datetime64[ns]
 10  salary               2020 non-null   int64         
 11  performance_score    2020 non-null   int64         
 12  psi_score            2020 non-null   int64         
 13  join_date            2020 non-nul

In [353]:
train_df.describe()

,id_employee,id_last_boss,seniority,office_distance,low_health_days,average_permanence,birth_date,salary,performance_score,psi_score,join_date,resign
count,2152.000000,2061.000000,2152.000000,2152.000000,2152.000000,2152.000000,2152,2.152000e+03,2084.000000,2152.000000,2152,2152.000000
mean,102070.160781,102090.326055,1.042286,3.111768,2.709108,6.644981,1982-08-05 21:32:07.137546432,4.570211e+05,63.472169,75.286245,2018-01-18 17:42:36.133828864,0.466078
min,100001.000000,102000.000000,1.000000,0.120000,0.000000,1.000000,1957-01-25 00:00:00,7.652100e+04,5.000000,58.000000,2012-01-04 00:00:00,0.000000
25%,101028.750000,102054.000000,1.000000,1.818750,1.000000,3.000000,1973-12-16 18:00:00,2.604398e+05,42.000000,71.000000,2014-12-28 12:00:00,0.000000
50%,102048.500000,102091.000000,1.000000,2.510000,2.000000,6.000000,1981-05-07 12:00:00,3.734225e+05,65.000000,75.000000,2018-02-18 00:00:00,0.000000
75%,103135.500000,102127.000000,1.000000,4.151250,3.000000,9.000000,1991-04-29 18:00:00,6.741930e+05,90.000000,79.000000,2021-02-23 00:00:00,1.000000
max,104171.000000,102172.000000,2.000000,21.050000,35.000000,25.000000,2003-02-19 00:00:00,1.900000e+06,99.000000,98.000000,2023-12-24 00:00:00,1.000000
std,1210.999342,43.586010,0.201288,1.789050,2.982975,4.283794,NaN,3.029435e+05,23.110594,6.050987,NaN,0.498964


In [354]:
test_df.describe()

,id_employee,id_last_boss,seniority,office_distance,low_health_days,average_permanence,birth_date,salary,performance_score,psi_score,join_date
count,2020.000000,1937.000000,2020.000000,2020.000000,2020.000000,2020.000000,2020,2.020000e+03,2020.000000,2020.00000,2020
mean,102103.807921,102089.646877,1.041089,3.059394,2.662376,6.713366,1982-04-05 03:26:43.960396032,4.638236e+05,89.146040,75.44505,2018-01-28 08:31:50.495049472
min,100000.000000,102000.000000,1.000000,0.110000,0.000000,1.000000,1956-08-16 00:00:00,7.551700e+04,80.000000,58.00000,2012-01-02 00:00:00
25%,101071.250000,102052.000000,1.000000,1.805000,1.000000,3.000000,1973-05-27 12:00:00,2.663110e+05,85.000000,71.00000,2014-12-14 00:00:00
50%,102117.500000,102089.000000,1.000000,2.542500,2.000000,6.000000,1981-03-23 00:00:00,3.747200e+05,90.000000,76.00000,2018-02-15 12:00:00
75%,103126.250000,102126.000000,1.000000,4.000000,3.000000,9.000000,1990-11-19 12:00:00,6.763075e+05,92.000000,79.00000,2021-03-25 06:00:00
max,104172.000000,102172.000000,2.000000,14.045000,23.000000,27.000000,2003-01-14 00:00:00,1.900000e+06,99.000000,98.00000,2023-12-24 00:00:00
std,1197.751058,43.691244,0.198545,1.637131,2.817351,4.307580,NaN,3.063601e+05,4.851837,6.00944,NaN


Observing the statistics summary, we can see that there are some outliers in `office_distance`, `low_health_days`, `average_permanence` and `salary` for both train and test data. They will be analyzed and actions will be taken during the EDA.

## Looking for typo errors in object data

In [355]:
train_df.describe(include='object')

,work_modality,gender,recruitment_channel,marital_estatus
count,2152,2152,2152,2152
unique,2,2,5,4
top,Presencial,Hombre,Portal Web,Soltero
freq,1530,1095,986,925


In [356]:
test_df.describe(include='object')

,work_modality,gender,recruitment_channel,marital_estatus
count,2020,2020,2020,2020
unique,2,2,5,4
top,Presencial,Mujer,Portal Web,Soltero
freq,1408,1037,869,800


In [357]:
# Checking work_modality categories

print(train_df.work_modality.unique())
print(test_df.work_modality.unique())

['Híbrida' 'Presencial']
['Híbrida' 'Presencial']


In [358]:
# Checking recruitment_channel categories

print(train_df.recruitment_channel.unique())
print(test_df.recruitment_channel.unique())

['Ferias & Networking' 'Referidos' 'Linkedin' 'Portal Web' 'Headhunter']
['Ferias & Networking' 'Portal Web' 'Linkedin' 'Referidos' 'Headhunter']


In [359]:
# Checking marital_estatus categories

print(train_df.marital_estatus.unique())
print(test_df.marital_estatus.unique())

['Soltero' 'Viudo' 'Divorciado' 'Casado']
['Soltero' 'Casado' 'Viudo' 'Divorciado']


From the data dictionary we can see that `seniority` and `id_last_boss` are categories too. However, id_ultimo_jefe has many categories, so it will be analyzed during the hypothesis testing phase.

In [360]:
cat_cols = ['work_modality', 'gender', 'recruitment_channel', 'marital_estatus', 'seniority']

# Defining a function to check categories proportion

def check_categories_proportion(df, col):
    
    """
    Calculate and print the proportion of each category in a specified column of a DataFrame.

    This function takes a pandas DataFrame and a column name, calculates the proportion 
    (as a percentage) of each unique value (category) in the specified column, and prints 
    the resulting proportions.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the data.
    col (str): The name of the column for which to calculate category proportions.

    Returns:
    None: This function prints the category proportions and does not return any value.

    Example:
    >>> import pandas as pd
    >>> data = {'category': ['A', 'B', 'A', 'C', 'B', 'A']}
    >>> df = pd.DataFrame(data)
    >>> check_categories_proportion(df, 'category')
    A    50.0
    B    33.3
    C    16.7
    Name: category, dtype: float64
    """
    
    proportion = df[col].value_counts(normalize=True) * 100
    print(proportion)

In [361]:
for col in cat_cols:
    check_categories_proportion(train_df, col)
    print('-'*40)

work_modality
Presencial    71.096654
Híbrida       28.903346
Name: proportion, dtype: float64
----------------------------------------
gender
Hombre    50.8829
Mujer     49.1171
Name: proportion, dtype: float64
----------------------------------------
recruitment_channel
Portal Web             45.817844
Linkedin               22.676580
Referidos              15.706320
Ferias & Networking    14.312268
Headhunter              1.486989
Name: proportion, dtype: float64
----------------------------------------
marital_estatus
Soltero       42.983271
Casado        38.708178
Divorciado    10.315985
Viudo          7.992565
Name: proportion, dtype: float64
----------------------------------------
seniority
1    95.771375
2     4.228625
Name: proportion, dtype: float64
----------------------------------------


In [362]:
for col in cat_cols:
    check_categories_proportion(test_df, col)
    print('-'*40)

work_modality
Presencial    69.70297
Híbrida       30.29703
Name: proportion, dtype: float64
----------------------------------------
gender
Mujer     51.336634
Hombre    48.663366
Name: proportion, dtype: float64
----------------------------------------
recruitment_channel
Portal Web             43.019802
Linkedin               25.247525
Referidos              16.336634
Ferias & Networking    14.059406
Headhunter              1.336634
Name: proportion, dtype: float64
----------------------------------------
marital_estatus
Soltero       39.603960
Casado        38.514851
Divorciado    12.425743
Viudo          9.455446
Name: proportion, dtype: float64
----------------------------------------
seniority
1    95.891089
2     4.108911
Name: proportion, dtype: float64
----------------------------------------


Categories proportion in both train and test data are similar.

## Extracting info from date columns

In [363]:
# Calculating age of join

train_df['join_age'] = (train_df['join_date'] - train_df['birth_date']).dt.days // 365

test_df['join_age'] = (test_df['join_date'] - test_df['birth_date']).dt.days // 365

In [364]:
# Obtaining season info

train_df['join_year'] = train_df['join_date'].dt.year
train_df['join_month'] = train_df['join_date'].dt.month

test_df['join_year'] = test_df['join_date'].dt.year
test_df['join_month'] = test_df['join_date'].dt.month

In [365]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152 entries, 0 to 2151
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_employee          2152 non-null   int64         
 1   id_last_boss         2061 non-null   float64       
 2   seniority            2152 non-null   int64         
 3   work_modality        2152 non-null   object        
 4   office_distance      2152 non-null   float64       
 5   low_health_days      2152 non-null   int64         
 6   gender               2152 non-null   object        
 7   recruitment_channel  2152 non-null   object        
 8   average_permanence   2152 non-null   int64         
 9   birth_date           2152 non-null   datetime64[ns]
 10  salary               2152 non-null   int64         
 11  performance_score    2084 non-null   float64       
 12  psi_score            2152 non-null   int64         
 13  join_date            2152 non-nul

In [366]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020 entries, 0 to 2019
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_employee          2020 non-null   int64         
 1   id_last_boss         1937 non-null   float64       
 2   seniority            2020 non-null   int64         
 3   work_modality        2020 non-null   object        
 4   office_distance      2020 non-null   float64       
 5   low_health_days      2020 non-null   int64         
 6   gender               2020 non-null   object        
 7   recruitment_channel  2020 non-null   object        
 8   average_permanence   2020 non-null   int64         
 9   birth_date           2020 non-null   datetime64[ns]
 10  salary               2020 non-null   int64         
 11  performance_score    2020 non-null   int64         
 12  psi_score            2020 non-null   int64         
 13  join_date            2020 non-nul

## Missing values treatment

From context of the business, is well known that people with performance_score greater than or equal to 80 is considered people with high perfomance, and everyone else is considered low performance. There are some missing values in this column on train data, and observing the statistics summary, most of the employees from the train data are low performance, while all the employees in the test data are high performance; this lead us to impute those missing values with 80, in order to help the people to generalize better the high performance employees. Additionally, performance categorical column will be added with 'high' for employees with performance score greater or equal to 80 and 'low' for everyone else.

In [367]:
train_df['performance_score'] = train_df['performance_score'].fillna(80)

Since id_ultimo_jefe is a categorical column, we can't impute it. So we gonna fill missing values with bfill method.

In [368]:
train_df['id_last_boss'] = train_df['id_last_boss'].fillna(method='bfill')

test_df['id_last_boss'] = test_df['id_last_boss'].fillna(method='bfill')

C:\Users\WILLIAM\AppData\Local\Temp\ipykernel_17532\2512515177.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_df['id_last_boss'] = train_df['id_last_boss'].fillna(method='bfill')
C:\Users\WILLIAM\AppData\Local\Temp\ipykernel_17532\2512515177.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_df['id_last_boss'] = test_df['id_last_boss'].fillna(method='bfill')


## Changing id_last_boss dtype

In [369]:
train_df['id_last_boss'] = train_df['id_last_boss'].astype(int)

test_df['id_last_boss'] = test_df['id_last_boss'].astype(int)

In [370]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152 entries, 0 to 2151
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_employee          2152 non-null   int64         
 1   id_last_boss         2152 non-null   int32         
 2   seniority            2152 non-null   int64         
 3   work_modality        2152 non-null   object        
 4   office_distance      2152 non-null   float64       
 5   low_health_days      2152 non-null   int64         
 6   gender               2152 non-null   object        
 7   recruitment_channel  2152 non-null   object        
 8   average_permanence   2152 non-null   int64         
 9   birth_date           2152 non-null   datetime64[ns]
 10  salary               2152 non-null   int64         
 11  performance_score    2152 non-null   float64       
 12  psi_score            2152 non-null   int64         
 13  join_date            2152 non-nul

In [371]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020 entries, 0 to 2019
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_employee          2020 non-null   int64         
 1   id_last_boss         2020 non-null   int32         
 2   seniority            2020 non-null   int64         
 3   work_modality        2020 non-null   object        
 4   office_distance      2020 non-null   float64       
 5   low_health_days      2020 non-null   int64         
 6   gender               2020 non-null   object        
 7   recruitment_channel  2020 non-null   object        
 8   average_permanence   2020 non-null   int64         
 9   birth_date           2020 non-null   datetime64[ns]
 10  salary               2020 non-null   int64         
 11  performance_score    2020 non-null   int64         
 12  psi_score            2020 non-null   int64         
 13  join_date            2020 non-nul

## Adding performance column

In [372]:
train_df['performance'] = ['high' if score >= 80 else 'low' for score in train_df['performance_score']]
test_df['performance'] = ['high' if score >= 80 else 'low' for score in test_df['performance_score']]

## Joining dataframes to create new features

Boss id are also present in employee id, so we can use this information to create new features that can help to improve the results

In [373]:
y = train_df['resign']

concat_df = pd.concat([train_df.drop(columns='resign'), test_df], axis=0).reset_index(drop=True)

In [374]:
expanded_df = pd.merge(concat_df, concat_df, how='left',
                             left_on='id_last_boss', right_on='id_employee',
                             suffixes=('_employee', '_boss'))

In [375]:
expanded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4172 entries, 0 to 4171
Data columns (total 38 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id_employee_employee          4172 non-null   int64         
 1   id_last_boss_employee         4172 non-null   int32         
 2   seniority_employee            4172 non-null   int64         
 3   work_modality_employee        4172 non-null   object        
 4   office_distance_employee      4172 non-null   float64       
 5   low_health_days_employee      4172 non-null   int64         
 6   gender_employee               4172 non-null   object        
 7   recruitment_channel_employee  4172 non-null   object        
 8   average_permanence_employee   4172 non-null   int64         
 9   birth_date_employee           4172 non-null   datetime64[ns]
 10  salary_employee               4172 non-null   int64         
 11  performance_score_employee    

In [384]:
# Dropping unnecessary features

expanded_df.drop(columns=['id_employee_boss', # Is the same as id_last_boss_employee
                          'seniority_boss', # It's 2 for all bosses
                          'birth_date_employee', # Age is already calculated
                          'birth_date_boss', # Age is already calculated
                          ], inplace=True)

## Creating new features

These new features allow us to make new features such as differences in numerical columns, determine if a employee joined after or before a boss and joined, etc

In [385]:
# Creating difference columns

expanded_df['office_distance_diff'] = expanded_df['office_distance_boss'] - expanded_df['office_distance_employee']
expanded_df['low_health_days_diff'] = expanded_df['low_health_days_boss'] - expanded_df['low_health_days_employee']
expanded_df['average_permanence_diff'] = expanded_df['average_permanence_boss'] - expanded_df['average_permanence_employee']
expanded_df['salary_diff'] = expanded_df['salary_boss'] - expanded_df['salary_employee']
expanded_df['join_days_diff'] = (expanded_df['join_date_boss'] - expanded_df['join_date_employee']).dt.days
expanded_df['joined_after_boss'] = [0 if value > 0 else 1 for value in expanded_df['join_days_diff']]
expanded_df['age_diff'] = expanded_df['join_age_boss'] - expanded_df['join_age_employee']

In [395]:
# Dropping join_date columns

expanded_df.drop(columns=['join_date_employee', 'join_date_boss'], inplace=True)

In [396]:
# Separating train and test df

train_final = pd.concat([expanded_df.loc[:2151, :], y], axis=1)
test_final = expanded_df.loc[2152:, :].reset_index(drop=True)

In [397]:
train_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152 entries, 0 to 2151
Data columns (total 40 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id_employee_employee          2152 non-null   int64  
 1   id_last_boss_employee         2152 non-null   int32  
 2   seniority_employee            2152 non-null   int64  
 3   work_modality_employee        2152 non-null   object 
 4   office_distance_employee      2152 non-null   float64
 5   low_health_days_employee      2152 non-null   int64  
 6   gender_employee               2152 non-null   object 
 7   recruitment_channel_employee  2152 non-null   object 
 8   average_permanence_employee   2152 non-null   int64  
 9   salary_employee               2152 non-null   int64  
 10  performance_score_employee    2152 non-null   float64
 11  psi_score_employee            2152 non-null   int64  
 12  marital_estatus_employee      2152 non-null   object 
 13  joi

In [398]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020 entries, 0 to 2019
Data columns (total 39 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id_employee_employee          2020 non-null   int64  
 1   id_last_boss_employee         2020 non-null   int32  
 2   seniority_employee            2020 non-null   int64  
 3   work_modality_employee        2020 non-null   object 
 4   office_distance_employee      2020 non-null   float64
 5   low_health_days_employee      2020 non-null   int64  
 6   gender_employee               2020 non-null   object 
 7   recruitment_channel_employee  2020 non-null   object 
 8   average_permanence_employee   2020 non-null   int64  
 9   salary_employee               2020 non-null   int64  
 10  performance_score_employee    2020 non-null   float64
 11  psi_score_employee            2020 non-null   int64  
 12  marital_estatus_employee      2020 non-null   object 
 13  joi

## Saving clean data

In [399]:
train_final.to_csv(paths.data_interim_dir('train_clean.csv'), index=False, sep=',')

test_final.to_csv(paths.data_interim_dir('test_clean.csv'), index=False, sep=',')